# Synaptipy — Writing a Custom Analysis Plugin

Synaptipy uses a **registry pattern** that lets you add new analysis
modules without modifying existing code.  Decorate a wrapper function
with `@AnalysisRegistry.register(...)` and Synaptipy’s Analyser tab
will auto-generate the GUI for it.

This notebook walks through the full process:

1. Understanding the registry decorator
2. Writing a minimal analysis function
3. Defining UI parameters and plot specs
4. Registering the function
5. Testing it headlessly

In [ ]:
import numpy as np
from typing import Dict, Any

# Import the full analysis package so existing decorators run
import Synaptipy.core.analysis  # noqa: F401 — triggers sub-module imports
from Synaptipy.core.analysis.registry import AnalysisRegistry

print(f"Registry loaded with {len(AnalysisRegistry.list_analysis())} analyses")

## 1. The Registry Pattern

Every registered analysis has:

| Field | Purpose |
|---|---|
| `name` | Unique key (e.g. `"spike_detection"`) |
| `label` | Human-readable tab title |
| `ui_params` | List of dicts that auto-build spin-boxes, check-boxes, drop-downs |
| `plots` | List of plot specifications (line traces, vlines, hlines, scatter) |

The wrapper function **must** have the signature:

```python
def my_wrapper(data, time, sampling_rate, **kwargs) -> Dict[str, Any]:
    ...
```

- `data`: 1-D numpy array (voltage or current trace)
- `time`: 1-D numpy array (time vector in seconds)
- `sampling_rate`: float (Hz)
- `**kwargs`: all UI parameter values, keyed by their `name`

The returned `dict` populates the results table.  Keys starting with `_`
are hidden from the table but available to plot specs.

## 2. Example: RMS Noise Analysis

Let’s write a plugin that computes the RMS noise and peak-to-peak
amplitude of a user-defined baseline window.

In [ ]:
def _compute_rms_noise(
    data: np.ndarray,
    time: np.ndarray,
    start: float,
    end: float,
) -> Dict[str, Any]:
    """Compute RMS noise and peak-to-peak amplitude in a time window.

    Parameters
    ----------
    data : np.ndarray
        Voltage trace (1-D).
    time : np.ndarray
        Time vector (seconds).
    start : float
        Window start (seconds).
    end : float
        Window end (seconds).

    Returns
    -------
    dict
        rms_noise_mv, peak_to_peak_mv, mean_mv, n_samples
    """
    mask = (time >= start) & (time <= end)
    segment = data[mask]

    if len(segment) == 0:
        return {"error": "Empty window"}

    mean_val = float(np.mean(segment))
    rms = float(np.sqrt(np.mean((segment - mean_val) ** 2)))
    ptp = float(np.ptp(segment))

    return {
        "rms_noise_mv": round(rms, 4),
        "peak_to_peak_mv": round(ptp, 4),
        "mean_mv": round(mean_val, 2),
        "n_samples": int(np.sum(mask)),
        # Hidden keys for plotting (prefixed with _)
        "_window_start": start,
        "_window_end": end,
        "_mean_line": mean_val,
    }

# Quick test with synthetic data
test_data = np.random.normal(-70, 0.3, 10000)
test_time = np.arange(10000) / 20000.0
result = _compute_rms_noise(test_data, test_time, 0.0, 0.1)
print(result)

## 3. Register the Plugin

Wrap the core function with the `@AnalysisRegistry.register` decorator.

### `ui_params` reference

Each parameter dict supports:

| Key | Type | Description |
|---|---|---|
| `name` | str | kwarg key passed to the wrapper |
| `label` | str | Label shown in the GUI |
| `type` | str | `"float"`, `"int"`, `"bool"`, `"choice"` |
| `default` | any | Default value |
| `min` / `max` | number | Range for spin-boxes |
| `step` | number | Spin-box step size |
| `choices` | list | Options for `"choice"` type |
| `visible_when` | dict | Conditional visibility: `{"param": "...", "value": "..."}` |

### `plots` reference

| Key | Description |
|---|---|
| `type` | `"trace"`, `"hlines"`, `"vlines"`, `"scatter"`, `"interactive_region"` |
| `data` | Result-dict key(s) supplying the data |
| `color` | Colour string |
| `label` | Legend label |

In [ ]:
@AnalysisRegistry.register(
    name="rms_noise",
    label="RMS Noise",
    ui_params=[
        {
            "name": "window_start",
            "label": "Window Start (s):",
            "type": "float",
            "default": 0.0,
            "min": 0.0,
            "max": 100.0,
            "step": 0.01,
        },
        {
            "name": "window_end",
            "label": "Window End (s):",
            "type": "float",
            "default": 0.1,
            "min": 0.0,
            "max": 100.0,
            "step": 0.01,
        },
    ],
    plots=[
        {
            "type": "interactive_region",
            "data": ["_window_start", "_window_end"],
            "color": "g",
        },
        {
            "type": "hlines",
            "data": ["_mean_line"],
            "color": "r",
            "label": "Mean",
        },
    ],
)
def run_rms_noise_wrapper(
    data: np.ndarray,
    time: np.ndarray,
    sampling_rate: float,
    **kwargs,
) -> Dict[str, Any]:
    """Registry wrapper for RMS noise analysis."""
    return _compute_rms_noise(
        data, time,
        start=kwargs.get("window_start", 0.0),
        end=kwargs.get("window_end", 0.1),
    )

print("Plugin registered!")
print(f"Registry now has {len(AnalysisRegistry.list_analyses())} analyses")

## 4. Verify Registration

List all registered analyses and confirm ours appears.

In [ ]:
for name, info in AnalysisRegistry.list_analyses().items():
    label = info.get("label", name)
    n_params = len(info.get("ui_params", []))
    print(f"  {label:35s}  ({n_params} params)")

## 5. Run the Plugin Headlessly

Call the wrapper directly (no GUI required) with synthetic data.

In [ ]:
# Generate a noisy trace
fs = 20_000
t = np.arange(0, 1.0, 1 / fs)
trace = -65.0 + np.random.normal(0, 0.25, len(t))

result = run_rms_noise_wrapper(
    data=trace,
    time=t,
    sampling_rate=fs,
    window_start=0.0,
    window_end=0.5,
)

print("Results:")
for k, v in result.items():
    if not k.startswith("_"):
        print(f"  {k}: {v}")

## 6. Deploying Your Plugin

To make your plugin available in the Synaptipy GUI:

1. Save your analysis function in a new file under
   `src/Synaptipy/core/analysis/` (e.g. `my_analysis.py`)
2. Add `from . import my_analysis` to
   `src/Synaptipy/core/analysis/__init__.py`
3. Re-install: `pip install -e .`
4. Launch Synaptipy — your new tab will appear automatically

### Important

Always import the **full package** (`import Synaptipy.core.analysis`)
rather than just the registry class.  Importing only
`from Synaptipy.core.analysis.registry import AnalysisRegistry`
does **not** trigger the sub-module decorators, so the registry
will appear empty.